In [2]:
import os
import sys
import pywFM
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dummyPy import OneHotEncoder

NumberOfRows = 10000
seed = 100

os.environ['LIBFM_PATH'] = os.path.expanduser('/home/jovyan/work/libfm/bin/')
filePath = os.getcwd() + '/yelp_dataset/'
print(filePath)
pd.set_option('display.max_columns', None)
%matplotlib inline

/home/jovyan/work/Personalization/yelp_dataset/


https://github.com/jfloff/pywFM

In [3]:
businuessColumns = ['state', 'business_id'] #'review_count', 'stars', 
userColumns = ["user_id", "average_stars", "review_count"] #, "elite", "compliment_photos", "compliment_list", "compliment_funny", "compliment_plain", "compliment_note", "compliment_writer", "compliment_cute", "compliment_more", "compliment_hot", "compliment_profile", "compliment_cool", "useful", "fans", "funny", "cool", 
ratingsColumns = ["business_id", "stars", "user_id"] #"cool", "funny", "useful", 
targetColumn = "stars"

In [4]:
os.listdir(filePath)

['.DS_Store',
 'yelp_ratings.csv',
 'textTransform.npz',
 'business_cleaned.csv',
 'review_cleaned_withTrainTest.csv',
 'user_cleaned.csv',
 'tfidfTransformer.pkl',
 '.ipynb_checkpoints',
 'countVectorizer.pkl']

In [5]:
business = pd.read_csv(filePath + "business_cleaned.csv", usecols=businuessColumns)
user = pd.read_csv(filePath + 'user_cleaned.csv', usecols=userColumns)
ratings = pd.read_csv(filePath + 'review_cleaned_withTrainTest.csv', nrows=NumberOfRows)

### Check duplicates and NaNs

In [6]:
ratings.drop_duplicates().shape, ratings.shape

((10000, 10), (10000, 10))

In [7]:
duplicateDF = ratings.loc[ratings[['user_id', 'business_id']].duplicated()]
x = ratings.loc[
    ratings["business_id"].isin(duplicateDF["business_id"]) & ratings["user_id"].isin(duplicateDF['user_id'])
].sort_values(["user_id", "business_id", "stars"])[["user_id", "business_id", "review_id", "date", "stars", "text"]]
print(x.shape)
x

(0, 6)


,user_id,business_id,review_id,date,stars,text


In [8]:
ratings.isnull().values.sum()

0

### Concatenate DataFrames

In [9]:
ratings = pd.merge(ratings, business, how='left', left_on='business_id', right_on='business_id')
ratings = pd.merge(ratings, user, how='left', left_on='user_id', right_on='user_id')

In [10]:
ratings['business_id'] = ratings['business_id'].astype('category')
ratings['business_id'] = ratings['business_id'].cat.codes

ratings['user_id'] = ratings['user_id'].astype('category')
ratings['user_id'] = ratings['user_id'].cat.codes

In [11]:
ratings.head()

,user_id,date,business_id,cool,funny,review_id,stars,text,useful,TrainTest,state,review_count,average_stars
0,1109,2015-01-01 00:00:30,5849,0.0,0.0,hNwleSHcvNksIQT31569Yg,5.0,Window washing inside and out done after many ...,5.0,1,NV,21,4.79
1,3246,2015-01-01 00:02:20,3694,0.0,0.0,DfZGAhAkPMJYDdXGRjhw8A,2.0,"I will admit, I do not have high expectations ...",0.0,0,AZ,30,3.80
2,5243,2015-01-01 00:02:35,2664,0.0,1.0,65viXwIysYSxEyPZgRSVbQ,1.0,The chicken curry I got was extremely dry. Des...,0.0,1,ON,19,2.68
3,29,2015-01-01 00:02:42,5565,0.0,0.0,hS3phsfoP-fAZVlMomx4Kg,5.0,Really took care of me on my trip out of state...,0.0,0,AZ,9,3.33
4,5941,2015-01-01 00:04:13,5235,1.0,0.0,eaDgBBrOtvFUto5pADZwQA,5.0,Gary and Chester run this 2 year old Church St...,1.0,0,ON,6,4.00


In [12]:
OneHotCols = ["user_id", "business_id", "state"]
encoder = OneHotEncoder(OneHotCols)
encoder.fit(ratings)
ratings = encoder.transform(ratings)

# def convertDummies(df, column_name):
#     df = pd.concat([
#         df.drop(column_name, axis=1),
#         pd.get_dummies(df[column_name], prefix=column_name+'_')
#     ], axis=1)
#     print(column_name, ' is ready.')
#     return df

In [13]:
ratings.drop(['text', 'review_id', 'date'], axis=1, inplace=True)

In [14]:
ratings.head()

,user_id_0,user_id_1,user_id_2,user_id_3,user_id_4,user_id_5,user_id_6,user_id_7,user_id_8,user_id_9,user_id_10,user_id_11,user_id_12,user_id_13,user_id_14,user_id_15,user_id_16,user_id_17,user_id_18,user_id_19,user_id_20,user_id_21,user_id_22,user_id_23,user_id_24,user_id_25,user_id_26,user_id_27,user_id_28,user_id_29,user_id_30,user_id_31,user_id_32,user_id_33,user_id_34,user_id_35,user_id_36,user_id_37,user_id_38,user_id_39,user_id_40,user_id_41,user_id_42,user_id_43,user_id_44,user_id_45,user_id_46,user_id_47,user_id_48,user_id_49,user_id_50,user_id_51,user_id_52,user_id_53,user_id_54,user_id_55,user_id_56,user_id_57,user_id_58,user_id_59,user_id_60,user_id_61,user_id_62,user_id_63,user_id_64,user_id_65,user_id_66,user_id_67,user_id_68,user_id_69,user_id_70,user_id_71,user_id_72,user_id_73,user_id_74,user_id_75,user_id_76,user_id_77,user_id_78,user_id_79,user_id_80,user_id_81,user_id_82,user_id_83,user_id_84,user_id_85,user_id_86,user_id_87,user_id_88,user_id_89,user_id_90,user_id_91,user_id_92,user_id_93,user_id_94,user_id_95,user_id_96,user_id_97,user_id_98,user_id_99,user_id_100,user_id_101,user_id_102,user_id_103,user_id_104,user_id_105,user_id_106,user_id_107,user_id_108,user_id_109,user_id_110,user_id_111,user_id_112,user_id_113,user_id_114,user_id_115,user_id_116,user_id_117,user_id_118,user_id_119,user_id_120,user_id_121,user_id_122,user_id_123,user_id_124,user_id_125,user_id_126,user_id_127,user_id_128,user_id_129,user_id_130,user_id_131,user_id_132,user_id_133,user_id_134,user_id_135,user_id_136,user_id_137,user_id_138,user_id_139,user_id_140,user_id_141,user_id_142,user_id_143,user_id_144,user_id_145,user_id_146,user_id_147,user_id_148,user_id_149,user_id_150,user_id_151,user_id_152,user_id_153,user_id_154,user_id_155,user_id_156,user_id_157,user_id_158,user_id_159,user_id_160,user_id_161,user_id_162,user_id_163,user_id_164,user_id_165,user_id_166,user_id_167,user_id_168,user_id_169,user_id_170,user_id_171,user_id_172,user_id_173,user_id_174,user_id_175,user_id_176,user_id_177,user_id_178,user_id_179,user_id_180,user_id_181,user_id_182,user_id_183,user_id_184,user_id_185,user_id_186,user_id_187,user_id_188,user_id_189,user_id_190,user_id_191,user_id_192,user_id_193,user_id_194,user_id_195,user_id_196,user_id_197,user_id_198,user_id_199,user_id_200,user_id_201,user_id_202,user_id_203,user_id_204,user_id_205,user_id_206,user_id_207,user_id_208,user_id_209,user_id_210,user_id_211,user_id_212,user_id_213,user_id_214,user_id_215,user_id_216,user_id_217,user_id_218,user_id_219,user_id_220,user_id_221,user_id_222,user_id_223,user_id_224,user_id_225,user_id_226,user_id_227,user_id_228,user_id_229,user_id_230,user_id_231,user_id_232,user_id_233,user_id_234,user_id_235,user_id_236,user_id_237,user_id_238,user_id_239,user_id_240,user_id_241,user_id_242,user_id_243,user_id_244,user_id_245,user_id_246,user_id_247,user_id_248,user_id_249,user_id_250,user_id_251,user_id_252,user_id_253,user_id_254,user_id_255,user_id_256,user_id_257,user_id_258,user_id_259,user_id_260,user_id_261,user_id_262,user_id_263,user_id_264,user_id_265,user_id_266,user_id_267,user_id_268,user_id_269,user_id_270,user_id_271,user_id_272,user_id_273,user_id_274,user_id_275,user_id_276,user_id_277,user_id_278,user_id_279,user_id_280,user_id_281,user_id_282,user_id_283,user_id_284,user_id_285,user_id_286,user_id_287,user_id_288,user_id_289,user_id_290,user_id_291,user_id_292,user_id_293,user_id_294,user_id_295,user_id_296,user_id_297,user_id_298,user_id_299,user_id_300,user_id_301,user_id_302,user_id_303,user_id_304,user_id_305,user_id_306,user_id_307,user_id_308,user_id_309,user_id_310,user_id_311,user_id_312,user_id_313,user_id_314,user_id_315,user_id_316,user_id_317,user_id_318,user_id_319,user_id_320,user_id_321,user_id_322,user_id_323,user_id_324,user_id_325,user_id_326,user_id_327,user_id_328,user_id_329,user_id_330,user_id_331,user_id_332,user_id_333,user_id_334,user_id_335,user_id_336,user_id_337,user_id_338,user_id_339,user_id_340,user_id_341,user_

### Factorization Machines 

In [15]:
train_X = ratings.loc[ratings['TrainTest'] == 1].drop(targetColumn, axis=1).drop('TrainTest', axis=1)
train_y = ratings.loc[ratings['TrainTest'] == 1][targetColumn]

test_X = ratings.loc[ratings['TrainTest'] == 0].drop(targetColumn, axis=1).drop('TrainTest', axis=1)
test_y = ratings.loc[ratings['TrainTest'] == 0][targetColumn]

In [16]:
train_X.shape, train_y.shape

((8554, 13570), (8554,))

In [17]:
test_X.shape, test_y.shape

((1446, 13570), (1446,))

In [18]:
fm = pywFM.FM(task='regression', k2=2, seed=seed)

In [19]:
model = fm.run(train_X.values, train_y.values, test_X.values, test_y.values)

12516


In [20]:
pd.concat([test_y.reset_index(drop=True), pd.Series(model.predictions, name='predicted')], axis=1).head()

,stars,predicted
0,2,3.78423
1,5,3.44281
2,5,4.61643
3,3,4.77653
4,2,3.21536


In [21]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(test_y.values, model.predictions))

1.3290130066205774

In [22]:
np.sqrt(mean_squared_error(test_y.values, np.tile(ratings['stars'].mean(), test_y.shape[0])))

1.5164853250228574